# Installing Hub

In [ ]:
!pip3 install hub --quiet
import os 
os.kill(os.getpid(), 9) 

     |████████████████████████████████| 261 kB 8.6 MB/s 
     |████████████████████████████████| 131 kB 60.1 MB/s 
     |████████████████████████████████| 1.8 MB 47.2 MB/s 
     |████████████████████████████████| 81 kB 7.0 MB/s 
     |████████████████████████████████| 5.8 MB 54.3 MB/s 
     |████████████████████████████████| 105 kB 46.6 MB/s 
     |████████████████████████████████| 55 kB 3.7 MB/s 
     |████████████████████████████████| 551 kB 73.0 MB/s 
     |████████████████████████████████| 256 kB 73.4 MB/s 
     |████████████████████████████████| 152 kB 67.6 MB/s 
     |████████████████████████████████| 3.0 MB 43.2 MB/s 
     |████████████████████████████████| 75 kB 4.5 MB/s 
     |████████████████████████████████| 111 kB 47.1 MB/s 
     |████████████████████████████████| 8.5 MB 43.4 MB/s 
     |████████████████████████████████| 79 kB 7.6 MB/s 
     |████████████████████████████████| 127 kB 74.6 MB/s 
     |████████████████████████████████| 41 kB 412 kB/s 
     |███████████████████

# Download raw dataset

In [1]:
from IPython.display import clear_output

In [ ]:
# Download Dataset here
!wget https://www.cse.cuhk.edu.hk/leojia/projects/hsaliency/data/ECSSD/images.zip
!wget https://www.cse.cuhk.edu.hk/leojia/projects/hsaliency/data/ECSSD/ground_truth_mask.zip
    
!unzip ground_truth_mask.zip 
!unzip images.zip

clear_output()

In [2]:
import hub
from PIL import Image 
import numpy as np 
import os
import pandas as pd
from matplotlib import pyplot as plt
from glob import glob
import time
from tqdm import tqdm

In [8]:
#@title Login to ActiveLoop

%env BUGGER_OFF=True
!activeloop login -u username -p password
!activeloop reporting --off

env: BUGGER_OFF=True
Successfully logged in to Activeloop.


# Creating dataset on hub

**Activeloop API** : https://docs.activeloop.ai/api-basics

In [10]:
# hubpath = './ecssd-hub'
hubpath = 'hub://activeloop/ecssd'
hub.delete(hubpath, large_ok=True)
ds = hub.dataset(hubpath)

start = time.time()

with ds:
    ds.create_tensor('images', htype='image', sample_compression='jpg')
    ds.create_tensor('masks', htype='binary_mask', sample_compression='lz4')
    
    for image in tqdm(os.listdir('images/')):
        file = image.split('.')[0]
        image_path = os.path.join('images', file+'.jpg')
        mask_path = os.path.join('ground_truth_mask', file+'.png')

        img_pil = Image.open(mask_path).convert('L')
        mask = np.array(img_pil, dtype='uint8')
        m, n = mask.shape
        mask = mask.reshape(m, n, 1)
        
        ds.append({
            'images' : hub.read(image_path),
            'masks' : mask.astype('bool')
        })
            
        
stop = time.time()
print(f"Time elapsed uploading : {stop-start}")

Your Hub dataset has been successfully created!
The dataset is private so make sure you are logged in!
This dataset can be visualized at https://app.activeloop.ai/activeloop/ecssd.


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 294.33it/s]


Time elapsed uploading : 146.83615732192993


# Testing dataset from hub

In [11]:
hubpath = 'hub://activeloop/ecssd'
ds = hub.load(hubpath)
print(ds.tensors)

hub://activeloop/ecssd loaded successfully.
This dataset can be visualized at https://app.activeloop.ai/activeloop/ecssd.
{'images': Tensor(key='images'), 'masks': Tensor(key='masks')}


In [12]:
index = 0
print(f"Image Shape: {ds.images[0].shape}")
print(f"Masks Shape: {ds.masks[index].shape}")

Image Shape: (328, 400, 3)
Masks Shape: (328, 400, 1)
